# 📊 Least-Square Fit to the Data by Polynomials

The data file [http://www.physics.drexel.edu/students/courses/physics-501/hw2.2.dat](http://www.physics.drexel.edu/students/courses/physics-501/hw2.2.dat) on the course Web page contains (hypothetical) experimental data on the measurement of a function $y(x)$. The $N$ data points are arranged, one measurement per line, in the format

$x_i \quad y_i \text{ (measured)} \quad \sigma_i$

where $\sigma_i$ is an estimate of the uncertainty in the $i$-th measurement. It is desired to find the least-square fit to the data by polynomials of the form:

$$y(x) = \sum_{j=1}^{m} a_j x^{j-1},$$

for specified values of $m$, by minimizing the quantity

$$\chi^2 = \sum_{i=1}^{N} \left( \frac{y_i - \sum_{j=1}^{m} a_j x_i^{j-1}}{\sigma_i} \right)^2.$$

As discussed in class (and in *Numerical Recipes*, pp. 671–676), write down the overdetermined design matrix equation that results from writing $y(x_i) = y_i$,

$$A \mathbf{a} = \mathbf{b}$$

where $A_{ij} = \frac{x_i^{j-1}}{\sigma_i}$, $b_i = \frac{y_i}{\sigma_i}$ (so the measurement uncertainties are included in each row), and $a$ is the vector of unknown coefficients. Solve this system using singular value decomposition (svdcmp in Numerical Recipes, svd in Python or Matlab) to obtain the best fitting polynomial for each of the cases $m = 2, 4, 7,$ and $13$. 
For each $m$, give the values of $a_j$ and $\chi^2$, and plot the data and the best fit on a single graph.

In [ ]:
from math import *
from numpy import *
from scipy import *

#Reading data from .dat file
infile = open('hw2.2.dat', 'r')
l = [lines for lines in infile.readlines()]
N = len(l)
k = 0
x = zeros(N)
y = zeros(N)
sigma = zeros(N)
for line in l:
    i = line.split()
    x[k] = i[0]
    y[k] = i[1]
    sigma[k] = i[2]
    k = k + 1
    
m = 13
A = zeros(N*m).reshape(N, m)
b = zeros(N)

#Defining design matrix A and target vector b
row = 0
while row < N:
    b[row] = y[row]/sigma[row]
    col = 0
    while col < m:
        A[row][col] = x[row]**(col+1-1)/sigma[row]
        col = col + 1
    row - row + 1
    
#Singular Value Decomposition
U, s, V = linalg.svd(A, full_matrices=False)

G = dot(U.T, b)
H = dot(diag(1/s), G)
astar = dot(V.T, H)

#Defining chi^2
D = dot(A, astar)
xsqrd = 0
k = 0
while k < N:
    xsqrd = xsqrd + (b[k]-D[k])**2
    k = k + 1
print(astar)
print(xsqrd)

#F-matrix that is similar to A but without 1/sigma_i
F = zeros(N*m).reshape(N, m)
row = 0
while row < N:
    col = 0
    while col < m:
        F[row][col] = x[row]**(col+1-1)
        col = col + 1
    row = row + 1
    
from matplotlib.pyplot import *

M = dot(F, astar)
xM = x
plot(xM, M, 'ro')
plot(x, y, '*')
show()